In [1]:
import openai


import os

openai.api_key = os.getenv("OPENAI_API_KEY")



## Get some RFC data

Took one from 2023 that's not included in the training data

In [2]:
# read the RFC file that's not in the

rfc_file = 'ietf.org_rfc_rfc9340.txt'

with open(rfc_file, 'r') as file:
    data = file.read().replace('\n', '')

# count words

words = data.split()
print('Number of words in text file :', len(words))



Number of words in text file : 14664


In [25]:
# Create an array of document with each element containing 100 lines

docs = []

with open(rfc_file, 'r') as file:
    doc = []
    for line in file:
        doc.append(line.strip())
        if len(doc) == 100:
            docs.append(doc)
            doc = []
    if len(doc) > 0:
        docs.append(doc)

len(docs)

# for each doc, join the lines to create a single string adding a \n between each line

for i in range(len(docs)):
    docs[i] = '\n'.join(docs[i])
    

# print the first lines of the first doc

print(docs[0][:100])







Network Working Group                                     T. Berners-Lee
Request for Comments:


In [26]:
# create a metadata in the form of a dictionary: {"source": "<docs_id>"}

metadata = []
for i in range(len(docs)):
    metadata.append({"source": i})

print(metadata[0:10])

[{'source': 0}, {'source': 1}, {'source': 2}, {'source': 3}, {'source': 4}, {'source': 5}, {'source': 6}, {'source': 7}, {'source': 8}, {'source': 9}]


In [27]:
# create a list of each ids as an array

ids = []
for i in range(len(docs)):
    ids.append(str(i))

# print the 10 first ids
print(ids[:10])

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


## Basic Storing of RFCs

Persistent ChromaDB

In [44]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="./chromadb/"
))

In [45]:
collection = client.create_collection(name="basic_rfc")

In [46]:
collection.add(
    documents=docs,
    metadatas=metadata,
    ids=ids
)

In [37]:
from pprint import pprint

results = collection.query(
    query_texts=["What is HTTP?"],
    n_results=10
)

pprint(results["distances"])
pprint(results["ids"])

[[0.5024073123931885,
  0.6518412232398987,
  0.6657850742340088,
  0.78008633852005,
  0.8532109260559082,
  1.0419760942459106,
  1.0496753454208374,
  1.0584864616394043,
  1.0728325843811035,
  1.1651463508605957]]
[['2', '0', '4', '8', '30', '3', '28', '7', '29', '13']]


In [47]:
pprint(docs[2])

('HTTP is also used as a generic protocol for communication between\n'
 'user agents and proxies/gateways to other Internet protocols, such as\n'
 'SMTP [12], NNTP [11], FTP [14], Gopher [1], and WAIS [8], allowing\n'
 'basic hypermedia access to resources available from diverse\n'
 'applications and simplifying the implementation of user agents.\n'
 '\n'
 '1.2  Terminology\n'
 '\n'
 'This specification uses a number of terms to refer to the roles\n'
 'played by participants in, and objects of, the HTTP communication.\n'
 '\n'
 'connection\n'
 '\n'
 'A transport layer virtual circuit established between two\n'
 'application programs for the purpose of communication.\n'
 '\n'
 'message\n'
 '\n'
 'The basic unit of HTTP communication, consisting of a structured\n'
 'sequence of octets matching the syntax defined in Section 4 and\n'
 'transmitted via the connection.\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Berners-Lee, et al           Informational                      [Page 4]\n'
 '\n'
 'RFC 1945   

## Thoughts

relative distance is indication of relevance?
doesn't seem normalized

